In [1]:
from tensorflow.keras.models import load_model
from flask import Flask, request, jsonify
import os
import numpy as np
import uuid

app = Flask(__name__) 

In [2]:
Expected = {
    "cylinders": {"min":3, "max": 8},
    "displacement": {"min":68.0, "max": 455.0},
    "horsepower": {"min":46.0, "max": 230.0},
    "weight": {"min":1613, "max": 5140},
    "acceleration": {"min":8.0, "max": 24.8},
    "model year": {"min":70, "max": 82},
    "origin": {"min":1, "max": 3}
}

In [3]:
model = load_model('Models/mpg_model.h5')

2021-09-27 18:10:40.249567: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX
2021-09-27 18:10:40.262409: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 3501000000 Hz
2021-09-27 18:10:40.263910: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fffccc324b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-09-27 18:10:40.263989: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-09-27 18:10:40.264349: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 25)                200       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                260       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 471
Trainable params: 471
Non-trainable params: 0
_________________________________________________________________


In [5]:
@app.route('/api', methods=['POST'])
def mpg_prediction():
    content = request.json
    errors = []
    for key in content:
        if key in Expected:
            expected_min = Expected[key]['min']
            expected_max = Expected[key]['max']
            value = content[key]
            if value < expected_min or value > expected_max:
                errors.append(f'Out of bounds: {key}, has value of: {value}, but it should be between {expected_min} and {expected_max}')
        else:
            errors.append(f'Unexpected field: {key}.')

    for key in Expected:
        if key not in content:
            errors.append(f'Missing value: {key}')
    if len(errors) < 1:
        x = np.zeros((1,7))

        x[0,0] = content['cylinders']
        x[0,1] = content['displacement']
        x[0,2] = content['horsepower']
        x[0,3] = content['weight']
        x[0,4] = content['acceleration']
        x[0,5] = content['model year']
        x[0,6] = content['origin']

        prediction = model.predict(x)
        mpg = float(prediction[0])
        response = { 'id': str(uuid.uuid4()), 'mpg': mpg, 'errors': errors}    
    else:
        response = {'id': str(uuid.uuid4()), 'errors': errors}
    
    return jsonify(response)

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
